# 使用深度学习模型进行估价预测

在这个 Notebook 中我们会介绍如何通过深度学习模型进行时间序列预测。通常来讲，像 ARIMA (Autoregressive Integrated moving average) 之类的传统算法往往并不能在股价预测的场景取得很好的效果。使用 RNN （循环神经网络）类深度学习算法会比传统的回归模型具有更好的效果。在这个环节中我们将专注于如何使用 SageMaker 实现预测类功能。

时间序列分析和预测是一个在现代科学中非常具有挑战的领域。在使用历史数据进行预测的时候，如果我们能事先知道影响未来状态的所有因素，就能够通过回归算法快速找到未来与这些影响因素之间的关系。但现实情况中，我们往往无法事先知道股价的所有影响因素。因此在预测问题中，往往使用一些与市场行情相关的银子，比如指数的表现、同行业其他股票的表现、市场情绪等因子。深度学习算法可以寻找这些因子之间的关联性，并且具有 LSTM 的模型可以识别一些股价的历史规律和周期性变化，更好地对未来状态进行预测。

在这个 Notebook 中，我们将使用白酒行业的几支股票数据（开盘价、收盘价、最高价、最低价）来训练模型用于对贵州茅台（600513）股价进行预测。通常的假设是，同一个行业板块中的几支股票价格往往具有一定的正相关性，同时使用这些关联数据进行模型训练往往可以获得更好的预测效果。首先我们会使用一个传统的 RNN 算法进行单支股票的训练和预测，随后将再尝试使用 DeepAR。

开始之前，先确保环境中有 Python 和 pip 。建议选择 Python 3 (Data Science) 内核。

## 使用自定义 RNN 模型进行股价预测

首先获取数据。在这个演示中，我们将使用最近的1000个交易日进行模型训练和预测。这里使用的是来自于 Keras 的 GRU layer（详情可以参照 train 脚本中的相关代码）。

### 准备工作

开始时需要 import 必要的模块，并且通过之前编写的 Athena 脚本进行数据调取：

In [ ]:
import boto3
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as lines
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import time
%matplotlib inline   

In [ ]:
import sys

directory = '/root/sagemaker-backtrader-examples'
if directory not in sys.path:
    sys.path.append(directory)
    
output_bucket = 'athena-output-cache' # 之前创建的 Athena 输出桶名

这个实验需要依赖 awscli、sagemaker、botocore 和 s3fs等库。在开始之前应当确认版本。这个实验默认应当使用 sagemaker 2.x 版本的 SDK 以及 0.4.2 版本的 s3fs。不同版本可能有不同的 botocore 依赖，请注意兼容性：

In [ ]:
!pip uninstall -y sagemaker
!pip install sagemaker==1.72.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install s3fs==0.4.2 -i https://pypi.tuna.tsinghua.edu.cn/simple

在之前的数据准备中，共有日线分钟线两种数据。以下代码适用于使用日线。在开始时我们需要定义一些参数：

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.predictor import RealTimePredictor
from sagemaker.predictor import csv_serializer

from ipywidgets import interact_manual, SelectionSlider
from IPython.display import display


role = sagemaker.get_execution_role()
session = sagemaker.Session()

# 参数定义

interval = 'D'

if interval == 'D':
    lag = 10 # 在预测时回顾的历史数据长度
    horizon = 5 # 在预测是前进的未来步数
    dateformat = '%Y-%m-%d'

target_stock = "600519" #  贵州茅台
covariate_stocks = "600559,000596,000568,000858,600779,002304" # 老白干、古井贡酒、泸州老窖、五粮液、水井坊、洋河股份
target_column = "close" # 需要预测的目标值
covariate_columns = "open,low,high"
num_epochs = 1000
percent_train = 85.0
num_units = 256
batch_size = 4096
dropout_ratio = 0.1

hyperparameters = {
    "interval": interval,
    "lag": str(lag),
    "horizon": str(horizon),
    "num_epochs": str(num_epochs),
    "batch_size": str(batch_size),
    "percent_train": str(percent_train),
    "num_units": str(num_units),
    "target_stock": target_stock,
    "covariate_stocks": covariate_stocks,
    "target_column": target_column,
    "covariate_columns": covariate_columns,
    "dropout_ratio": str(dropout_ratio)
}

### 训练数据

接下来，首先需要定义数据存储的位置：

In [ ]:
# 定义训练数据位置

s3 = boto3.client('s3')

role = sagemaker.get_execution_role()
session = sagemaker.Session()
s3_bucket = session.default_bucket()

aws_default_region = session.boto_session.region_name
aws_account_id = session.boto_session.client('sts').get_caller_identity()['Account']
image_repo_name = 'sagemaker-backtrader-examples'
image_tag = '2_prediction'
model_name = 'custom-rnn'

base_job_name = "{}-{}-{}".format(image_repo_name, interval, model_name)
data_location = "s3://{}/{}/{}/{}/{}/data/stockdata.csv".format(s3_bucket, image_repo_name, image_tag, model_name, interval)
output_location = "s3://{}/{}/{}/{}/{}/output".format(s3_bucket, image_repo_name, image_tag, model_name, interval)
print('任务名：', base_job_name)
print('数据：', data_location)
print('输出：', output_location)

再将数据从 Athena 调取并储存到 S3：

In [ ]:
from data_util import get_query_result

database = 'stock-data-demo'
table = 'stock_day'
fields = 'ticker,tradedate,openprice,closeprice,highestprice,lowestprice,turnovervol,accumadjfactor'
end_date = '2020-4-13'
orderby = 'tradedate'
limit = '1000'

ticker = target_stock
query_string = f'''
SELECT DISTINCT {fields}
FROM "{database}"."{table}"
WHERE ticker='{ticker}'
AND tradedate<='{end_date}'
ORDER BY {orderby}
DESC
LIMIT {limit}
'''

df = get_query_result(query_string, output_bucket)

df['ticker'] = df['ticker'].apply(lambda x: str(x))
df['ticker'] = df['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
df['openprice'] = df['openprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['closeprice'] = df['closeprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['highestprice'] = df['highestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['lowestprice'] = df['lowestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df.set_index('tradedate', inplace=True)
df.sort_index(0, inplace=True)
df.drop('accumadjfactor', 1, inplace=True)
df.rename(columns={'openprice': 'open'}, inplace=True)
df.rename(columns={'closeprice': 'close'}, inplace=True)
df.rename(columns={'highestprice': 'high'}, inplace=True)
df.rename(columns={'lowestprice': 'low'}, inplace=True)
df.rename(columns={'turnovervol': 'volume'}, inplace=True)

start_date = df.index[0]
end_date = df.index[-1]
print('Target stock:', ticker, start_date, '-', end_date)

stock_data = pd.DataFrame([])
stock_data = pd.concat([stock_data, df])

covariates = covariate_stocks.split(',')
for ticker in covariates:

    query_string = f'''
    SELECT DISTINCT {fields}
    FROM "{database}"."{table}"
    WHERE ticker='{ticker}'
    AND tradedate>='{start_date}'
    AND tradedate<='{end_date}'
    ORDER BY {orderby}
    ASC
    '''

    df = get_query_result(query_string, output_bucket)

    df['ticker'] = df['ticker'].apply(lambda x: str(x))
    df['ticker'] = df['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
    df['openprice'] = df['openprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df['closeprice'] = df['closeprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df['highestprice'] = df['highestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df['lowestprice'] = df['lowestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df.set_index('tradedate', inplace=True)
    df.sort_index(0, inplace=True)
    df.drop('accumadjfactor', 1, inplace=True)
    df.rename(columns={'openprice': 'open'}, inplace=True)
    df.rename(columns={'closeprice': 'close'}, inplace=True)
    df.rename(columns={'highestprice': 'high'}, inplace=True)
    df.rename(columns={'lowestprice': 'low'}, inplace=True)
    df.rename(columns={'turnovervol': 'volume'}, inplace=True)

    print('Covriate stock:', ticker, df.index[0], '-', df.index[-1])
    stock_data = pd.concat([stock_data, df])

stock_data.to_csv(data_location, index=True)
stock_data.head()

接下来，定义之前从 CodeBuild 搭建的镜像名称：

In [ ]:
# 定义镜像

image = '{}.dkr.ecr.{}.amazonaws.com.cn/{}:{}'.format(aws_account_id, aws_default_region, image_repo_name, image_tag)
print('镜像：', image)

### 训练自定义算法模型

要在Amazon SageMaker中训练模型，您需要创建训练任务。在以下的示例中，我们将使用 SageMaker 的 Estimator API 进行操作。训练完成后，SageMaker会将代码存储在容器中路径 /opt/ml/model 下的所有文件存档，并使这些文件打包在名为 model 的 .tar.gz 文件上传至 S3 的指定位置。

在算法中，我们自动将数据集进行了 training 和 testing 的划分。同时也记录了模型训练的 loss history，可以在最终上传至 S3 的 tar 文件进行查看。

接下来我们将通过以下代码开始模型的训练，训练过程将经历 1000 个 epoch：

In [ ]:
# 创建容器镜像和训练实例
rnn = Estimator(image, role, 1, 'ml.m5.4xlarge',
                output_path=output_location,
                base_job_name=base_job_name,
                sagemaker_session=session)

rnn.set_hyperparameters(**hyperparameters)

# 模型训练
rnn.fit(data_location)

# 定义路径
estimator_job = rnn.latest_training_job.job_name
model_archive = "{}/{}/{}/{}/output/{}/output/model.tar.gz".format(image_repo_name, image_tag, model_name, interval, estimator_job)
print("Estimator created at completion of training job {}".format(estimator_job))
print('模型路径：', model_archive)

训练完成后可以选择从 S3 中对模型训练的 loss history 进行简单的可视化。可以看到，loss 随着 epoch 增加呈稳定下降的趋势：

In [ ]:
import custom_rnn_util as util

util.plot_loss(s3, s3_bucket, model_archive, "loss_history.csv")

### 模型部署

模型训练成功后，可以选择将模型部署到实例进行使用。部署时需要选择适当的实例大小，以下示例将通过 SageMaker SDK 来完成，也可以选择在 SageMaker Studio 或 SageMaker 控制台中手动操作完成：

In [ ]:
# Create an endpoint on a web server
predictor = rnn.deploy(1, 'ml.m5.large', serializer=csv_serializer)

### 使用模型进行预测和可视化

如果上述的部署顺利完成，就可以创建 predictor 进行推理了。在训练代码中，我们已经将数据集分成了 training 和 test。我们首先将从训练结果的 artifact 中加载这些数据集，并且对预测的结果和真实的历史数据进行比较。

In [ ]:
# 创建 predictor
predictor = RealTimePredictor(estimator_job, content_type = "text/csv")

接下来我们将 S3 中的测试数据集解压缩，并且获取数据集的路径：

In [ ]:
# 加载 test 数据集
filepaths = util.extract_matching_csv_files_from_s3_tarfile(s3, s3_bucket, model_archive, "test[0-9]+.csv", model_archive[:model_archive.find("/model")], 0)
print('数据路径：')
for f in filepaths:
    print(f)

testfilename = "testdata.csv"
testdata = util.load_csv_from_s3_tarfile(s3, s3_bucket, model_archive, testfilename, model_archive[:model_archive.find("/model")], 0)
testdata['ticker'] = testdata['ticker'].apply(lambda x: str(x))
testdata['ticker'] = testdata['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
print(testdata.shape)
testdata.head()

接下来将 S3 中的测试数据路径传递给 predictor 类进行时序预测：

首先，我们将测试数据的 S3 路径传递给 predictor 进行测试，然后简单比较一下预测值和实际值：

In [ ]:
util.plot_sample_predictions(predictor, filepaths, target_stock, target_column, lag)

在另一个测试中，我们可以看一下阶梯式的预测结果，并且进行画图：

In [ ]:
util.plot_sample_test_performance(predictor, testdata, target_stock, covariate_stocks, target_column, lag, horizon, horizon)

您也可以使用另一个 utility 函数对所有的测试进行可视化：

In [ ]:
util.plot_overall_test_performance(predictor, testdata, target_stock, covariate_stocks, target_column, lag, horizon, horizon)

### 小结

可以看出，股价预测模型在一些交易日中可以取得比较好的效果，但并不是总能取得很理想的效果。如果有兴趣的话，可以尝试增加样本大小，或者更换一些股票进行尝试。

稍后我们将使用 AWS 自带的 DeepAR 进行新的尝试。

### 删除资源

实验完成后，请记得删除部署的节点，以避免产生额外费用。这个步骤也可以在控制台中完成：

In [ ]:
predictor.delete_endpoint()

## 使用 DeepAR 进行股价预测

像以前一样，我们将使用白酒行业的股票，将这些系列的收盘价作为将要预测的时间序列建立模型。但是，这里的区别在于，我们不必指定哪个是主序列，哪些是外生序列。DeepAR 算法致力于建立一个统一的模型，该模型将考虑所有提供的时间序列中的趋势，并可以针对任何这些趋势生成预测。这将避免需要针对不同的时间序列构建不同的模型。

正如在自定义 RNN 示例中使用协变量时间序列一样，我们可以通过使用 Dynamic Feature 在 DeepAR 上起到类似的效果。DeepAR 还可以通过整数定义不同股票的类型。在股价预测的场景中，您可以将不同行业的股票都一同进行训练。假设这些行业板块之间确实存在某种可靠的正相关/负相关的关联性，DeepAR 就可以学习和捕捉这些关系，这往往可以提升模型的准确性。

接下来的步骤将使用 AWS 托管的 DeepAR 镜像进行模型训练和部署。

### 准备工作

同样的，我们需要对实验所需的环境和依赖包进行初始化：

In [ ]:
import boto3
import os
import datetime
import sagemaker
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox
import matplotlib
import deepar_util as util

In [ ]:
import sys

directory = '/root/sagemaker-backtrader-examples'
if directory not in sys.path:
    sys.path.append(directory)
    
output_bucket = 'athena-output-cache' # 之前创建的 Athena 输出桶名

In [ ]:
!pip show awscli sagemaker botocore

In [ ]:
!pip install s3fs==0.4.2 -i https://pypi.tuna.tsinghua.edu.cn/simple

在之前的自定义 RNN 示例中，我们实现了训练代码，因此可以使用超参数来自定义训练的某些方面。对于DeepAR，就像任何AWS提供的算法一样，您可以使用类似的hyparparameters配置来充分利用模型。

DeepAR 允许您控制神经网络体系结构，例如网络的层数、每层中的循环单元数层、学习率，预测长度等。需要注意的是，更高的预测长度会消耗相当多的内存资源，因此建议您适当对超参进行选择。在以下的示例中，我们配置了一些超参，以便在合理的时间内完成训练。你可能注意到有些超参和之前的自定义 RNN 模型超参略有不同。

关于超参方面的秒数可以参照：https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html

In [ ]:
# 参数定义

interval = 'D'

if interval == 'D':
    prediction_length = 91  
    context_length = 91

target_stock = "600519" #  贵州茅台
covariate_stocks = "600559,000596,000568,000858,600779,002304" # 老白干、古井贡酒、泸州老窖、五粮液、水井坊、洋河股份
tickers = [target_stock] + covariate_stocks.split(',')
target_column = "close"
covariate_columns = "open,low,high"

covariates = covariate_stocks.split(',')
target_column = 'close'
covariate_columns = covariate_columns.split(',')

train_test_split = 0.8
num_test_windows = 4

hyperparameters = {
    "prediction_length": str(prediction_length), # number of time-steps model is trained to predict, always generates forecasts with this length
    "context_length": str(context_length), # number of time-points that the model gets to see before making the prediction, should be about same as the prediction_length
    "time_freq": interval, # granularity of the time series in the dataset
    "epochs": "200", # maximum number of passes over the training data
    "early_stopping_patience": "40", #training stops when no progress is made within the specified number of epochs
    "num_layers": "2", #number of hidden layers in the RNN, typically range from 1 to 4    
    "num_cells": "40", #number of cells to use in each hidden layer of the RNN, typically range from 30 to 100
    "mini_batch_size": "128", #size of mini-batches used during training, typically values range from 32 to 512
    "learning_rate": "1e-3", #learning rate used in training. Typical values range from 1e-4 to 1e-1
    "dropout_rate": "0.1", # dropout rate to use for regularization, typically less than 0.2. 
    "likelihood": "gaussian" # noise model used for uncertainty estimates - gaussian/beta/negative-binomial/student-T/deterministic-L1
}

metrics=[]
metrics.extend(covariate_columns)
metrics.append(target_column)
print('Metrics:', metrics)

定义训练模型所需的 IAM 角色、镜像名称、S3输入和输出路径等：

In [ ]:
s3 = boto3.client('s3')

role = sagemaker.get_execution_role()
session = sagemaker.Session()
s3_bucket = session.default_bucket()

aws_default_region = session.boto_session.region_name
aws_account_id = session.boto_session.client('sts').get_caller_identity()['Account']
image_repo_name = 'sagemaker-backtrader-examples'
image_tag = '2_prediction'
model_name = 'deepar'

# Obtain container image URI for SageMaker-DeepAR algorithm, based on region
region = session.boto_region_name
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
print("Model will be trained using container image : {}".format(image_name))

# Define training data location
base_job_name = "{}-{}-{}".format(image_repo_name, model_name, interval)
s3_data_key = "{}/{}/{}/{}/{}/data".format(s3_bucket, image_repo_name, image_tag, model_name, interval)
s3_output_path = "s3://{}/{}/{}/{}/{}/output".format(s3_bucket, image_repo_name, image_tag, model_name, interval)

print('镜像：', image_name)
print('数据：', s3_data_key)
print('输出：', s3_output_path)

### 训练数据

与自定义 RNN 模型类似，我们先调取 1000 个交易日的数据进行训练：

In [ ]:
from data_util import get_query_result

database = 'stock-data-demo'
table = 'stock_day'
fields = 'ticker,tradedate,openprice,closeprice,highestprice,lowestprice,turnovervol,accumadjfactor'
end_date = '2020-4-13'
orderby = 'tradedate'
limit = '1000'

ticker = target_stock
query_string = f'''
SELECT DISTINCT {fields}
FROM "{database}"."{table}"
WHERE ticker='{ticker}'
AND tradedate<='{end_date}'
ORDER BY {orderby}
DESC
LIMIT {limit}
'''

df = get_query_result(query_string, output_bucket)

df['ticker'] = df['ticker'].apply(lambda x: str(x))
df['ticker'] = df['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
df['openprice'] = df['openprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['closeprice'] = df['closeprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['highestprice'] = df['highestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df['lowestprice'] = df['lowestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
df.set_index('tradedate', inplace=True)
df.sort_index(0, inplace=True)
df.drop('accumadjfactor', 1, inplace=True)
df.rename(columns={'openprice': 'open'}, inplace=True)
df.rename(columns={'closeprice': 'close'}, inplace=True)
df.rename(columns={'highestprice': 'high'}, inplace=True)
df.rename(columns={'lowestprice': 'low'}, inplace=True)
df.rename(columns={'turnovervol': 'volume'}, inplace=True)

print("Target stock:", ticker)

stock_data = pd.DataFrame([])
stock_data = pd.concat([stock_data, df])
start_date = df.index[0]
end_date = df.index[-1]

for ticker in covariates:

    query_string = f'''
    SELECT DISTINCT {fields}
    FROM "{database}"."{table}"
    WHERE ticker='{ticker}'
    AND tradedate>='{start_date}'
    AND tradedate<='{end_date}'
    ORDER BY {orderby}
    ASC
    '''

    df = get_query_result(query_string, output_bucket)

    df['ticker'] = df['ticker'].apply(lambda x: str(x))
    df['ticker'] = df['ticker'].apply(lambda x: '0'*(6-len(x)) + x)
    df['openprice'] = df['openprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df['closeprice'] = df['closeprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df['highestprice'] = df['highestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df['lowestprice'] = df['lowestprice'] * df['accumadjfactor'] / df['accumadjfactor'].iloc[-1]
    df.set_index('tradedate', inplace=True)
    df.sort_index(0, inplace=True)
    df.drop('accumadjfactor', 1, inplace=True)
    df.rename(columns={'openprice': 'open'}, inplace=True)
    df.rename(columns={'closeprice': 'close'}, inplace=True)
    df.rename(columns={'highestprice': 'high'}, inplace=True)
    df.rename(columns={'lowestprice': 'low'}, inplace=True)
    df.rename(columns={'turnovervol': 'volume'}, inplace=True)

    print('Covriate stock:', ticker, df.index[0], '-', df.index[-1])
    
    stock_data = pd.concat([stock_data, df])

df = util.normalize_data(stock_data, target_column, covariate_columns)
print(df.shape)
df.head()

将训练数据进行格式转换并输出到 S3：

In [ ]:
train_data, test_data, train_start_date, train_end_date = util.generate_train_test_set(df, target_column, covariate_columns, interval, train_test_split, num_test_windows)

train_channel = util.write_dicts_to_file(train_data, s3_data_key, "train")
test_channel = util.write_dicts_to_file(test_data, s3_data_key, "test")
print("Train channel {}".format(train_channel))
print("Test channel {}".format(test_channel))

如果有兴趣，在开始训练之前可以简单看一下时间序列的情况：

In [ ]:
util.metrics_plot(tickers, metrics, df)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (25, 17) # use bigger graphs
util.timeseries_plot(tickers, metrics, df)

### 模型训练

与使用自定义 RNN 模型类似，接下来的代码将使用 SageMaker SDK 在一台新实例上开始模型的训练工作。根据使用的实例大小不同，这可能会花费10-15分钟左右的时间：

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.4xlarge',
    base_job_name=base_job_name,
    output_path=s3_output_path
)

# Set the hyperparamters
estimator.set_hyperparameters(**hyperparameters)

# Specify data channels
data_channels = {
    "train": train_channel,
    "test": test_channel
}

# Train the model
estimator.fit(inputs=data_channels, wait=True)

#Wait for training to finish
estimator_job = estimator.latest_training_job.job_name
print("Estimator created at completion of training job {}".format(estimator_job))

### 模型部署

同样的，在模型训练成功后，我们需要将训练的结果部署到 SageMaker 终端节点上进行使用。

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    predictor_cls=util.DeepARPredictor)
print("Pedictor attached to Endpoint: {}".format(predictor.endpoint))

### 使用 DeepAR 进行预测

部署成功后，我们可以直接运行以下的代码完成预测工作：

In [ ]:
predictor = util.DeepARPredictor(estimator_job)

In [ ]:
ts, dynamic_feat, observed = util.query_for_stock('600519', target_column, covariate_columns, df, prediction_length)
print('Historical times:', ts.index[0], '-', ts.index[-1], 'total length', len(ts.index))
prediction_index = df.index.tolist()
prediction_start = prediction_index.index(ts.index[-1]) + 1
prediction_end = prediction_index.index(ts.index[-1]) + 1 + prediction_length
prediction_index = prediction_index[prediction_start:prediction_end]
print('Prediction times:', prediction_index[0], '-', prediction_index[-1], 'total length', len(prediction_index))

quantiles = [0.10, 0.50, 0.90]
prediction = predictor.predict(ts, prediction_index, dynamic_feat=dynamic_feat, quantiles=quantiles, return_samples=False)

可以通过画图对预测结果进行可视化。需要注意的是，DeepAR 产生的结果是对未来进行的概率预测。图中的阴影部分代表 90% 和 10% quantile 覆盖的范围：

In [ ]:
util.plot_predicted_observed_at_quantile(ts, observed, prediction, quantile='0.5', upper='0.9', lower='0.1')

### 结语

DeepAR 可以视作对自 RNN 的一种改进。这种跨系列训练比 ARIMA 模型更有助于我们改善预测，但前提是我们需要收集足够的市场数据，并进行标注和分类。

在最开始的数据准备工作中，我们已经提供了全市场一定时间范围内的日线和分钟线历史行情数据。您可以自行在这些示例代码的基础上进行各种模型的实验和研究。请注意我们提供的数据是从公开来源收集的，可能无法完全保证数据质量。您也可以选择上传属于自己的数据集并进行 ETL 后通过 Athena 调取使用。

在以上的预测中我们试用了目标系列的收盘价作为目标值，开盘价、最高价和最低价作为动特征。这种应用方法在某种程度上是人为的，因为实际上在收盘之前就取得准确的价格数据是不可能的。在实际的应用场景中您可能需要基于其他因子进行预测，这些因子的来源可能包括：

 - 公司未来的销售/收入目标
 - 分析师股价指导
 - 宏观经济预测
 - 通过期权所得到的价格和波动性预测

### 删除资源

实验完成后，请记得删除部署的节点，以避免产生额外费用。这个步骤也可以在控制台中完成：

In [ ]:
predictor.delete_endpoint()